# 트리의 앙상블
## 앙상블 학습(Ensemble Training)이 무엇인지 이해하고 다양한 앙상블 학습 알고리즘을 배운다.

### Data 의 종류
- 정형 데이터(Structured data) 와 비정형 데이터(Unstructured data)
- 정형 데이터의 대표적인 예시는 csv, database, excel file들이다. 각 Feature 들이 Labeled 되어있음.
- 비정형 데이터는 엑셀로 표현하기 어려운 것들이다.
- 예를 들어 글, 사진, 디지털 음악 등등

- 앙상블 학습 (Ensemble Learning): **정형 데이터**를 가장 잘 다루는 알고리즘
- 이 알고리즘은 대부분 Decision Tree 기반으로 만들어져있음.

## Ensemble Learning 1 : Random Forest

- Decision Tree 를 **Random**하게 만들어 Decision Tree 로 이루어진 Forest 를 만든다.
- 그리고 각 결정 트리의 Prediction 을 종합해 Final Prediction 을 제시한다.

- Random Forest 는 각 Tree 를 훈련하기 위한 데이터를 Random 하게 만드는데 이 방식이 독특함.
- 우리가 입력한 Train_set 에서 Randomly Extract & 복원 추출하여 Train_set 을 만든다.
- 복원 추출하기 때문에 중복된 데이터들이 함유될 수 있음.
- 이렇게 만들어진 샘플을 **Bootstrap Sample** 이라고 함.
- 일반적으로 Bootstrap Sample 의 크기는 Train_input data 의 크기와 동일하게 만들어짐.

- 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾습니다.
- 분류 모델이 RandomForestClassifier 는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택합니다.
- 즉 4개의 특성이 있다면 노드마다 2개를 랜덤하게 선택하여 사용합니다.
- 다만 회귀 모델인 RandomForestRegressor 는 전체 특성을 사용합니다.

- sklearn 의 RandomForest 는 기본적으로 100개의 Decision Tree 를 이런 방식으로 훈련합니다.
- Classification : 각 Tree 의 Class별 확률을 평균내서 가장 높은 확률을 가진 Class 를 예측으로 제공합니다.
- Regression : 단순히 각 Tree 의 예측을 평균냅니다.

**Random Forest 의 장점**
- 랜덤하게 선택된 Sample 과 Feature 을 사용하기 떄문에 Train_set 에 과대적합되는 것을 막아준다.
- 따라서 Valid_set 과 Test_set 에서 안정적인 성능을 얻을 수 있다.

## Code

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy() # 여러개 선택하기 때문에 이중 리스트처럼 입력
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data,target, test_size = 0.2, random_state = 42)

In [3]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs = -1, random_state=42)
scores = cross_validate(rfc, train_input, train_target, return_train_score = True, n_jobs = -1)
scores

{'fit_time': array([0.38122702, 0.38105392, 0.27015042, 0.12557435, 0.38054013]),
 'score_time': array([0.07901382, 0.04752827, 0.01569104, 0.01601744, 0.01586747]),
 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]),
 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}

In [6]:
print(f"평균 train_score : {np.mean(scores['train_score']):.4f}, 평균 test_score :  {np.mean(scores['test_score']):.4f}")

평균 train_score : 0.9974, 평균 test_score :  0.8905


- Random Forest 는 Decision Tree 의 Ensemble 이기 때문에 DecisionTreeClassifier 가 제공하는 중요한 Parameter 들을 모두 제공합니다.
- criterion, max_depth, max_features, min_samples_split, min_impurity_decrease, min_samples_leaf 등등
- 또한 Decision Tree 의 큰 장점 중 하나인 "특성 중요도"를 계산해줍니다.
- Random Forest의 특성 중요도는 각 Decision Tree 의 중요도를 취합한 것입니다.

### Feature Importance (특성 중요도) 출력해보기

In [7]:
rfc.fit(train_input, train_target)
rfc.feature_importances_
# [알코올 도수, 당도, pH]

array([0.23167441, 0.50039841, 0.26792718])

### RandomForestClassifier 의 또 다른 기능 : 자체적으로 모델을 평가하는 점수를 제공함.

- RandomForest 에서 Train_set 을 형성할 때, 중복을 허용하여 만들기 때문에 선택되지 못한 data set 이 존재함.
- 이를 OOB Sample (Out Of Bag) 이라고 한다.
- 이 Sample 을 Valid_set 처럼 사용하여 평가함.

In [8]:
rf = RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42)
rf.fit(train_input,train_target)
print(rf.oob_score_)

0.8934000384837406


## Ensemble Learning 2 : Extra Trees

### Classification

- RandomForest 와 유사한 알고리즘이지만, 차이점은 Bootstrap sample 을 사용하지 않는다는 점에서 차이가 있다.
- Decision Tree 를 만들때 그냥 바로 전체 Train_set 을 사용한다.
- 대신 Node 를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할한다.
- 즉 splitter = 'random' 인 결정 트리를 생성함.
- 하나의 Decision Tree 에서 Feature 을 무작위로 분할한다면 성능이 낮아지겠지만, 많은 Tree 를 ensemble(엮기) 하기 때문에 overfitting 을 막고 Valid set score 을 높이는 효과가 있다.

In [9]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

In [10]:
print(f"평균 train_score : {np.mean(scores['train_score']):.4f}, 평균 test_score :  {np.mean(scores['test_score']):.4f}")

평균 train_score : 0.9975, 평균 test_score :  0.8888


- Random Forest 와 비슷한 결과를 얻었으나, 이는 특성이 많지 않아서 큰 차이가 나타나지 않은 것임.
- Random Forest 보다 더 많은 Decision Tree 를 훈련해야 하나, Random 하게 Node 를 분할하기 때문에 계산이 빠르다!

In [15]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


### Regression
- ExtraTreesClassifier 의 회귀 버전은 ExtraTreesRegressor

## Ensemble Learning 3 : Gradient Boosting

- 깊이가 얕은 Decision Tree 를 사용하여 이전 Tree 의 오차를 보완하는 방식으로 앙상블하는 방법
- sklearn 이 제공하는 GradientBoostingClassifier 는 기본적으로 깊이가 3인 트리를 100개 사용
- 깊이가 얕은 Tree 를 이용하기 때문에 Overfitting 에 강하고, 높은 일반화 성능을 기대할 수 있다.
- Classification : Logistic Loss function 사용, Regression : Mean Square Error function 사용

### Gradient 의 의미

- 경사 하강법을 사용하여 트리를 앙상블에 추가한다.
- 손실 함수를 산으로 정의하고 가장 낮은 곳을 찾아 내려오는 과정. 
- 모델의 coef 와 intercept 를 조금씩 바꿔가며 최소 손실 지점을 찾는 것.
- Gradient Boosting  기법은 결정 트리를 계속 추가해가면서 점차 낮은 곳을 찾아 이동한다.
- 점차 = 깊이가 낮은 트리

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)

In [18]:
print(f"평균 train_score : {np.mean(scores['train_score']):.4f}, 평균 test_score :  {np.mean(scores['test_score']):.4f}")

평균 train_score : 0.8881, 평균 test_score :  0.8720


거의 overfitting 되지 않았음.
Gradient Boosting 은 Tree 의 수를 늘려도 overfitting 에 강하다.
Learning_Rate(학습률) 을 증가시키고 Tree 의 개수를 늘리면 성능이 향상된다.

In [19]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

In [20]:
print(f"평균 train_score : {np.mean(scores['train_score']):.4f}, 평균 test_score :  {np.mean(scores['test_score']):.4f}")

평균 train_score : 0.9465, 평균 test_score :  0.8780


In [21]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68011572 0.16116151]


## Ensemble Learning 4 : Histogram-based Gradient Boosting

- 정형 데이터를 다루는 ML 알고리즘 중에 가장 인기가 높은 알고리즘
- input feature 값을 256 개의 구간으로 분할. -> Node 를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.

- HistGradientBoostingClassifier 에는 Tree 의 개수를 지정하는데 n_estimators 대신에 Boosting 반복 횟수를 지정하는 max_iter 을 사용한다.
- 성능을 높이려면 max_iter 매개변수를 테스트해보는게 좋다.

In [22]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score = True)

In [23]:
print(f"평균 train_score : {np.mean(scores['train_score']):.4f}, 평균 test_score :  {np.mean(scores['test_score']):.4f}")

평균 train_score : 0.9322, 평균 test_score :  0.8801


**HbGB 의 특성중요도를 계산하는 함수 : permutation_importance()**
- 특성을 하나씩 Random 하게 섞어서 모델 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산한다.
- 섞는 횟수 매개변수 : n_repeats, default 값 = 5

In [24]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input,train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats = 10, random_state=42, n_jobs=-1)

In [31]:
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


Test_set 에서의 Feature Importance

In [29]:
result = permutation_importance(hgb, test_input, test_target, n_repeats = 10, random_state=42, n_jobs=-1)

In [32]:
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [33]:
hgb.score(test_input, test_target)

0.8723076923076923

시중 모델
1. XGBoost
2. LightGBM